# Machine Learning Systems Homework 1

<a target="_blank" href="https://colab.research.google.com/github/mlsyscourse/homework1/blob/main/mlsys_hw1.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Homework due: Feb 12, 2024, 11:59 pm, Eastern Time**.

Automatic differentiation is the foundation technique of training a machine learning model.
In this homework, you will implement a simple prototype automatic differentiation system (learned in lecture 4), build up your own logistics regression model, and train the model on a handwritten digit dataset.

* You should work on this homework **individually** -- it is not a team homework.
* This homework does not require GPU. You can do the homework on either Google Colab (by clicking the badge above), your laptop/desktop, or any server that you have access to.
* This homework is pure Python. No C++ is needed in this homework.
* Please check out the end of this notebook for the homework submission requirement.
* Please do not share your solution on publicly available websites (e.g., GitHub).
* **About testing and grading.** We provide a set of public test cases (under `tests/` directory) for you to test your homework implementation. **Meanwhile, we have a set of private test cases to grade your homework.** The scores you get in each task is proportional to the number of private test cases you pass. You can submit your homework for multiple times, and only the last submission will be graded after the homework due. **This means you will not be able to see your scores immediately after submission.** We encourage you to create your own test cases, which helps you confirm the correctness of your code.

## Preparation

* If you are using Google Colab environment, please make a copy of this notebook file by selecting "Save a copy in Drive" from the "File" menu, and then run the code block below to set up workspace. After cloning, you will see the cloned repository in the "Files" bar on the left.

In [1]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p 15442
%cd /content/drive/MyDrive/15442
!git clone https://github.com/mlsyscourse/homework1.git
%cd /content/drive/MyDrive/15442/homework1

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
/content/drive/MyDrive/15442
Cloning into 'homework1'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 105 (delta 52), reused 105 (delta 52), pack-reused 0
Receiving objects: 100% (105/105), 274.29 KiB | 3.97 MiB/s, done.
Resolving deltas: 100% (52/52), done.
/content/drive/MyDrive/15442/homework1


* If you are using local/server environment, please clone this repository.

```shell
git clone https://github.com/mlsyscourse/homework1.git
cd homework1
export PYTHONPATH=.:$PYTHONPATH
```

## Part 1: Automatic Differentiation Framework (60 pt)

In part 1, you will implement the reverse mode automatic differentiation algorithm.

The auto diff algorithm in this homework works on a **computational graph**.
A computational graph describes the process of computation of an expression.
For example, given $x_1$, $x_2$, the expression $y = x_1 \times x_2 + x_1$ has the following computational graph:

<img src="https://raw.githubusercontent.com/mlsyscourse/homework1/main/figure/computational_graph.jpg" alt="figure/computational_graph.jpg" width="60%"/>

Let's first walk you through the basic concepts and data structures in the framework.
A computational graph consists of **nodes**, where each node denotes an intermediate step of computation during computing the entire expression.
Every node is composed of the three parts (`auto_diff.py` line 6):

- an **operation** (field `op`), which defines the operation that the node computes.
- a list of **input nodes** (field `inputs`), which indicates the input source of the computation.
- optionally, additional "**attributes**" (field `attrs`). The attributes that a node has depends on the op of the node. We will explain the attributes later in this part.

We can define an input node of a computational graph with `ad.Variable`. For example, the input variable nodes $x_1$ and $x_2$ can be defined as

```python
import auto_diff as ad

x1 = ad.Variable(name="x1")
x2 = ad.Variable(name="x2")
```

In `auto_diff.py` (line 81), you can see that the essence of `ad.Variable` is to construct a node
with op `placeholder` and the given name. The input nodes have empty `inputs` and `attrs`:
```python
class Variable(Node):
    def __init__(self, name: str) -> None:
        super().__init__(inputs=[], op=placeholder, name=name)
```

Here, the `placeholder` defines the computation of a input variable node, which is "doing nothing."
Besides `placeholder`, we have other ops defined in `auto_diff.py`. For example,

- op `add` defines the addition of two nodes,
- op `matmul` defines the matrix multiplication of two nodes.

Notably, these ops are globally defined for only once, and the `op` field of every node is such
a globally defined op.

Now, back to our example of $y = x_1 \times x_2 + x_1$.
Now that we have `x1` and `x2` as two input variable nodes, we can define the rest of the computational graph
with a one-line Python code:
```python
y = x1 * x2 + x1
```

This line first constructs a node with op `mul` (multiplication) and `x1`, `x2` as `inputs`,
and then constructs a node with op `add` which takes the previous multiplication node and `x1` as `inputs`.
As a result, our computational graph contains four nodes in the end.

It worths noting that a computational graph (e.g., the four nodes we defined) **does not** carry concrete values of nodes.
The style of this homework is consistent with the TensorFlow v1 style, introduced in the lecture.
This is different from frameworks like PyTorch, where the values of input tensors will be given in the beginning,
and the values of intermediate tensors are eagerly computed along the way when those tensors are defined.
In our computational graph, to compute the value of output `y` given values of input `x1`, `x2`,
we provide the `Evaluator` class (`auto_diff.py` line 373).

Here is an example of how `Evaluator` works. The constructor of `Evaluator` takes a list of nodes to evaluate.
By writing
```python
evaluator = ad.Evaluator(eval_nodes=[y])
```
it means that we construct an `Evaluator` instance which aims to compute the value of `y`.
Then we provide the values (assuming all `numpy.ndarray` in this homework) of the input tensors through the main interface `Evaluator.run` (which you need to implement):
```python
import numpy as np

x1_value = np.array(2)
x2_value = np.array(3)
y_value = evaluator.run(input_dict={x1: x1_value, x2: x2_value})
```

At a high level, here the `run` method consumes the input values via a dictionary `Dict[Node, numpy.ndarray]`,
computes the value of node `y` internally, and returns the result.
Given `2 * 3 + 2 = 8`, the returned `y_value` should be `np.ndarray(8)` eventually (of course, it will not return the correct value before your finish implementing):
```python
np.testing.assert_allclose(y_value, np.array(8))
```

`Evaluator.run` method effectively computes the forward computation of nodes, and we can go ahead to talk about the backward.
As you learned in the lecture, in order to compute the output gradient with regard to each input node in a computational graph,
we can extend the forward graph with the additional backward part.
Once we have the forward and backward graph together, by given input node values,
we can use `Evaluator` to compute the output value, loss value, and the gradient values of each input nodes altogether with a single-time `Evaluator.run`.

The function `gradients(output_node: Node, nodes: List[Node]) -> List[Node]` in `auto_diff.py` is
the function you need to implement to construct the backward graph.
This function takes an output node (usually the node of the loss function in machine learning), whose gradient is treated as 1,
takes the list of nodes to compute gradients for,
and returns the gradient nodes with regard to each node in the input list.

Back to our example, after implementing `gradients`, you can run
```python
x1_grad, x2_grad = ad.gradients(output_node=y, node=[x1, x2])
```
to get the gradients of $y$ regarding $x_1$ and $x_2$ respectively.
And you can construct `Evaluator` on nodes `y`, `x1_grad` and `x2_grad`, and use `Evaluator.run`
to compute the output value and input gradients.

Finally, before leaving the homework to you, we introduce how op works.
As you can find in `auto_diff.py`, each op defines three methods:

- `__call__(self, **kwargs) -> Node`, which takes in the input nodes (and attributes), constructs a new node with this op, and returns the constructed node.
- `compute(self, node: Node, input_values: List[np.ndarray]) -> np.ndarray`, which takes the node to compute with its input values, and returns the computed value of the given node.
- `gradient(self, node: Node, output_grad: Node) -> List[Node]`, which takes a node and the gradient node of this node, and returns the nodes of the input partial adjoints (one for each input node).

In general, the `Op.compute` method computes the value of a single node with given node inputs, and `Evaluator.run` function computes the value of a graph output with given graph inputs.
`Op.gradient` method constructs the backward computational graph for a single node, and `gradients` function constructs the backward graph for a graph.
That being said, your implementation of `Evaluator.run` should effectively make use of the `compute` method of op,
and likewise, the `gradients` function implementation should leverage the `gradient` method defined in op.

### Your tasks

**Task 1 (10 pt).** Implement the `compute` method for all ops in `auto_diff.py`. We provide the examples of `AddOp` and `AddByConstOp`, and you need to implement the rest.
For this homework, you can assume that the inputs of addition/multiplication/division have the same shape.

We provide sample tests in `tests/test_auto_diff_node_forward.py`.
You can test your task 1 implementation by running

In [2]:
!python3 -m pytest -l -v tests/test_auto_diff_node_forward.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/15442/homework1
plugins: anyio-3.7.1
collected 8 items                                                                                  

tests/test_auto_diff_node_forward.py::test_mul PASSED                                        [ 12%]
tests/test_auto_diff_node_forward.py::test_mul_by_const PASSED                               [ 25%]
tests/test_auto_diff_node_forward.py::test_div PASSED                                        [ 37%]
tests/test_auto_diff_node_forward.py::test_div_by_const PASSED                               [ 50%]
tests/test_auto_diff_node_forward.py::test_matmul[False-False] PASSED                        [ 62%]
tests/test_auto_diff_node_forward.py::test_matmul[False-True] PASSED                         [ 75%]
tests/test_auto_diff_nod

**Task 2 (15 pt).** Implement the `Executor.run` method in `auto_diff.py`.
You may want to get the [topological sort](https://en.wikipedia.org/wiki/Topological_sorting) of the computational graph
in order to compute the output value.

We provide sample tests in `tests/test_auto_diff_graph_forward.py`.
You can test your task 2 implementation by running

In [3]:
!python3 -m pytest -l -v tests/test_auto_diff_graph_forward.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/15442/homework1
plugins: anyio-3.7.1
collected 12 items                                                                                 

tests/test_auto_diff_graph_forward.py::test_identity PASSED                                  [  8%]
tests/test_auto_diff_graph_forward.py::test_add PASSED                                       [ 16%]
tests/test_auto_diff_graph_forward.py::test_add_by_const PASSED                              [ 25%]
tests/test_auto_diff_graph_forward.py::test_mul PASSED                                       [ 33%]
tests/test_auto_diff_graph_forward.py::test_mul_by_const PASSED                              [ 41%]
tests/test_auto_diff_graph_forward.py::test_div PASSED                                       [ 50%]
tests/test_auto_diff_gra

**Task 3 (15 pt).** Implement the `gradient` method for all ops in `auto_diff.py`. We provide the examples of `AddOp` and `AddByConstOp`, and you need to implement the rest.

We provide sample tests in `tests/test_auto_diff_node_backward.py`.
You can test your task 3 implementation by running

In [4]:
!python3 -m pytest -l -v tests/test_auto_diff_node_backward.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/15442/homework1
plugins: anyio-3.7.1
collected 7 items                                                                                  

tests/test_auto_diff_node_backward.py::test_mul PASSED                                       [ 14%]
tests/test_auto_diff_node_backward.py::test_div PASSED                                       [ 28%]
tests/test_auto_diff_node_backward.py::test_div_by_const PASSED                              [ 42%]
tests/test_auto_diff_node_backward.py::test_matmul[False-False] PASSED                       [ 57%]
tests/test_auto_diff_node_backward.py::test_matmul[False-True] PASSED                        [ 71%]
tests/test_auto_diff_node_backward.py::test_matmul[True-False] PASSED                        [ 85%]
tests/test_auto_diff_nod

**Task 4 (20 pt).** Implement `gradients` function in `auto_diff.py`.
You may also find topological sort helpful in the implementation.

We provide sample tests in `tests/test_auto_diff_graph_backward.py`.
You can test your task 4 implementation by running

In [5]:
!python3 -m pytest -l -v tests/test_auto_diff_graph_backward.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/15442/homework1
plugins: anyio-3.7.1
collected 3 items                                                                                  

tests/test_auto_diff_graph_backward.py::test_graph PASSED                                    [ 50%]
tests/test_auto_diff_graph_backward.py::test_gradient_of_gradient PASSED                     [100%]

======================================== 3 passed in 0.28s =========================================


### A few notes

1. **Zero-rank arrays in NumPy.** As mentioned earlier, all values are assumed to have type `numpy.ndarray` throughout this homework. One thing you may find interesting about NumPy is, if we add two zero-rank arrays together (e.g., `np.array(1) + np.array(2)`), it results in a scalar value, rather than a zero-rank array:
```
>>> x = np.array(1)
>>> y = np.array(2)
>>> type(x), type(y), x.ndim, y.ndim
(<class 'numpy.ndarray'>, <class 'numpy.ndarray'>, 0, 0)
>>> z = x + y
>>> z, type(z)
(3, <class 'numpy.int64'>)
```
This means that, if you want to have a rigorous implementation of your homework, you need to check the result type
at the end of `compute` methods, and wraps any scalar values back to `numpy.ndarray`.
However, for simplicity, we do not requrire you to do this, and it is completely up to you:
there will be no test for this behavior, and you won't get fewer credits because of not doing this.
Python by default also does not have eager type checking to throw error when you do not handle the scalars.

2. **`Node.attrs`.** In the reference implementation of `AddByConstOp` in `auto_diff.py`, you will find that the `attrs` field is used to store the constant operand of the addition in the returned node. In general, the `attrs` field of a node stores all the **constants** that are known when constructing the computational graph: for the case of `AddByConstOp`, the constant operand is stored as a node attribute. While for general cases, an attribute does not have to be a node operand. You can see that in `MatMulOp` in `auto_diff.py`, we store the boolean flags denoting whether to transpose the input matrices as attributes. In the next part of this homework, you may implement op like `SumOp`, and find it useful to store the axis being reduced as a node attribute.

3. **Minimality of `gradients`.** The `gradients` function constructs the backward graph and returns the gradient nodes with regards to required nodes. One interesting note here is the minimality of the constructed backward graph. For example, for a graph of `y = x1 * x2 + x1`, if we are only interested in the gradient of `x1 * x2`, a minimal backward graph only contains the gradient node of `x1 * x2`, which means it is not necessary to construct the gradient nodes for `x1` and `x2`. In this homework, we **do not** require you to construct the minimal backward graph, but it would be a good mental exercise to think about the possible pros/cons of constructing minimal backward graphs.

## Part 2: SGD for logistic regression (40 pt)

In this part, you need to implement the stochastic gradient descent (SGD) algorithm to train a simple logistic regression model.

Specifically, for input $x\in \mathbb{R}^n$ , we'll consider a logistic regression model of the form
$$z = W^T x+b$$
where $W\in \mathbb{R}^{n\times k}, b\in \mathbb{R}^k$ represent the weight and bias of modethe model, and $z\in \mathbb{R}^k$ represents the logits output by the network.

The model should be trained with softmax / cross-entropy loss on mini-batches of training data,
which means we want to solve the following optimization problem, under the mini-batch setting.
\begin{equation}
\min_{W, b} \;\; \ell_{\mathrm{softmax}}(XW+b, y),
\end{equation}
where $X\in \mathbb{R}^{b \times n}$.


### Your tasks

In general, you need the following steps (components) to train the logistic regression model:

**Task 5 (15 pt).** Define the forward computational graph for $Z = XW+b$ in `logistic_regression` function in `logistic_regression.py`.
Note that $XW$ is a 2-dim matrix, while $b$ is a 1-dim vector.
You may find it helpful to introduce a new operator that broadcasts the $b$ vector to the matrix shape (think about why?).
In common frameworks, people use `broadcast_to`, e.g., [NumPy](https://numpy.org/doc/stable/reference/generated/numpy.broadcast_to.html),
for this purpose. However, our computational graph nodes do not carry shape information.
So you may want to slightly tweak the interface of your own broadcasting op.

**Task 6 (15 pt).** Implement `softmax_loss` function in `logistic_regression.py` that constructs the computational graph of softmax loss.
The softmax loss takes an input node of logits and a node of one-hot encodings of the true labels.
As a reminder, for a multi-class output that can take on values $y \in \{1,\ldots,k\}$,
the softmax loss takes a vector of logits $z \in \mathbb{R}^k$ and the true class $y \in \{1,\ldots,k\}$ (which is encoded for this function as a one-hot vector),
and returns a loss defined by
\begin{equation}
\ell_{\mathrm{softmax}}(z, y) = \log\sum_{i=1}^k \exp z_i - z_y.
\end{equation}
You may need to introduce new operators to compute summation, logarithm, exponentiation and their gradients to build up softmax loss function.

**Task 7 (10 pt).** Implement `sgd_epoch` function in `logistic_regression.py` to run a single epoch of SGD.
In this function, you need to split the input data and labels into several mini-batches.
Then run the constructed computational graph given one batch as input.
Collect gradients and update the weight/bias of your logistic regression model correspondingly.

If your implementation is correct, you will observe that the prediction accuracy on the handwritten digit dataset is around 95% by running `logistic_regression.py`:
```shell
> python3 logistic_regression.py
...
Final test accuracy: 0.9611111111111111
```

In [ ]:
!python3 logistic_regression.py

**Hint.** When you find the current op set not satisfying your needs, consider introducing a new op.

## Part 3. Create Your Own Test Cases (0 pt)

We encourage you to create your own test cases, which helps you confirm the correctness of your implementation.
If you are interested, you can write your own tests in `tests/test_customized_cases.py` and share them with us by including this file in your submission.
We appreciate it if you can share your tests, which can help improve this course and the homework. Please note that this part is voluntary.

## Part 4. Homework Feedback (0 pt)

This is the first time we offer this course, and we appreciate any homework feedback from you.
You can leave your feedback (if any) in `feedback.txt`, and submit it together with the source code.
Possible choices can be:

- How difficult do you think this homework is?
- How much time does the homework take? Which task takes the most time?
- Which part of the homework do you feel hard to understand?
- And any other things you would like to share.

Your feedback will be very useful in helping us improve the homework quality
for next years.


## How to Submit Your Homework

In the home directory for the assignment, execute the command

In [ ]:
!make handin.tar

This will create an archive file with `auto_diff.py`, `logistic_regression.py`, `tests/test_customized_cases.py` and `feedback.txt`.
You can check the contents of `handin.tar` to make sure it contains all the needed files:

In [ ]:
!tar -tf handin.tar

It is expected to list the four files:
```
auto_diff.py
logistic_regression.py
feedback.txt
tests/test_customized_cases.py
```

Then, please go to Autolab at https://autolab.andrew.cmu.edu/courses/15442-s24 and submit the file `handin.tar`.

This assignment is not automatically graded, and you will not receive immediate feedback upon submission in Autolab.
You can submit multiple times, but only your final submission will be graded, and the time stamp of that submission will be used in determining any late penalties.
If you are enrolled in the course (on SIO), but not registered on Autolab and/or Gradescope, please let the course staff know in a private post on Piazza.